In [143]:
import requests, csv, io
from datetime import datetime as dt
from pprint import pprint

In [128]:
sheet_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRrTyONku0NgOZ2f_aFeJ8bKJ4K4wc4brHxMm9Q1_Si72bhdgefUinLCiUmoZFQg4V7F95Y4rbU0bcf/pub?gid=0&single=true&output=tsv'

In [129]:
response = requests.get(sheet_url)

In [130]:
response.encoding = response.apparent_encoding # Fix encoding (see https://stackoverflow.com/questions/44203397/python-requests-get-returns-improperly-decoded-text-instead-of-utf-8)

In [131]:
response.raise_for_status()

In [132]:
headers = response.text.split("\r\n")[0]
headers = headers.split("\t")
rows = response.text.split("\r\n")[1:]

In [133]:
_rows = []
for row in rows:
    fields = row.split("\t")
    if fields[0] == "" or fields[0] == None: continue
    _ = {}
    for i, field in enumerate(fields):
        _[headers[i]] = field
    _rows.append(_)
rows = _rows

In [145]:
import re
for i, row in enumerate(rows):
    sources = row['Source/s'].split("\r\n")
    for source in sources:
        row['Source/s'] = row['Source/s'].replace("Archive: ", "")

        title, vol, iss, date = None, None, None, None
        g = re.search("^([A-Za-z\s.-]+) \(.*\); (.*): ", row['Source/s'])
        if g:
            title = g.groups()[0]

            g2 = re.search("(Vol\. (\d+))", row['Source/s'])
            if g2: vol = g2.groups()[0]

            g3 = re.search("(Iss\. (\d+))", row['Source/s'])
            if g3: iss = g3.groups()[0]

            g4 = re.search("((\d{2}) (Jan|Feb|Mar|Apr|May|Jun|June|Jul|July|Aug|Sep|Oct|Nov|Dec) (\d{4}))", row['Source/s'])
            if g4: 
                date_str = g4.groups()[0].replace("June", "Jun")
                date_str = date_str.replace("July", "Jul")
                date = dt.strptime(date_str, "%d %b %Y")
            else: 
                g4b = re.search("((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (\d+), (\d{4}))", row['Source/s'])
                if g4b: date = dt.strptime(g4b.groups()[0], "%b %d, %Y")

            if date is None:
                print(f"Warning: Could not find date in the following data.\n\n{row['Source/s']}\n\n\n")
        g = re.search("(.*): (.*)\.", row['Source/s'])
        if g:
            page_number = g.groups()[1]

        '''
        rows[i]['title'] = title
        rows[i]['vol'] = vol
        rows[i]['iss'] = iss
        rows[i]['date'] = date
        '''


Current Literature (1888-1912); New York Vol. VOL. XX,, Iss. No. 5, (Nov 1896): 452.




The American Law Review (1866-1906); St. Louis Vol. 38,  (Sep/Oct 1904): 751. 





In [146]:
rows

[{'id': '2',
  'Year': '1854',
  'First name': 'Thomas L.',
  'Last name': 'Floyd',
  'alt spelling of name': 'Thomas Floyd; Thomas L. Flloyd; Thomas L. Floydd; Thomas Flloyd; Thomas Floydd; Tomas Floyd; Tomas L. Floyd',
  'Company': "Kunkel's Nightingale Minstrels",
  'Play/performance (mentioned)': '',
  'Keyword': '',
  'Comment': '(The celebrated female impersonator of Kunkel’s Nightingale Opera Troupe. Eventually co-manager with George Kunkel of Marshall Theatre, Richmond, VA. Born 1828 = 26 years old here..  George Kunkel was famous "Ethiopian minstrel" - from 1845 the head of the Nightingale Troupe (after George W. Harvey) - immense reputation in a few years - enabling Kunkel with Mr Ford to "embark extensively in theatrical enterprises" - Washington (D.C.) Theatre was his second acquisition - Moxley was manager here, "known as Master Floyd or \'Miss Fanny\'." - "Mr. Geo. Kunkel\'s Engagement Abroad, A Representative Minstrel\'s Eventful Career," The Sun (1837-1992); Baltimore, 

In [117]:
dt.strptime("06 Aug 1905", "%d %b %Y")

datetime.datetime(1905, 8, 6, 0, 0)